In [68]:
import scipy
import numpy as np
import torch
from torch import nn

In [4]:
data = scipy.io.loadmat("data/datafile.mat")

In [26]:
n_train = data["D"].item()
n_test = data["T"].item()
word_types = data["TW"].item()
features_size = data["M"].item()
P = 30 # size of hidden layer for RNN

In [62]:
H_train = torch.tensor([[idx.item() if idx.shape == (1, 1) else 0 for idx in sentence] for sentence in data["H_train"]])
H_test = torch.tensor([[idx.item() if idx.shape == (1, 1) else 0 for idx in sentence] for sentence in data["H_test"]])

labels_train = torch.tensor([[idx.item() if idx.shape == (1, 1) else 0 for idx in sentence] for sentence in data["labels_train"]])
labels_test = torch.tensor([[idx.item() if idx.shape == (1, 1) else 0 for idx in sentence] for sentence in data["labels_test"]])

In [81]:
class SentenceAnalysis(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(
            input_size=1, hidden_size=features_size, num_layers=1
        )
        self.linear = nn.Linear(features_size, word_types)
    
    def forward(self, x):
        x_ = self.rnn(x)
        logits = self.linear(x_[0])

        return logits

In [119]:
model = SentenceAnalysis()
lr = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()
epochs = 4000

In [120]:
for epoch in range(epochs):
    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []
    model.train()
    for x, y in zip(H_train, labels_train):
        final_len = (x == 2).nonzero().squeeze().item()+1

        x = x[:final_len].float().unsqueeze(-1)
        y_ = y[:final_len].long()-1

        logits = model(x)

        loss_out = loss(logits, y_)
        loss_out.backward()
        optimizer.step()

        class_hat = torch.round(nn.functional.softmax(logits, dim=-1))
        classes = nn.functional.one_hot(y_)

        errors = (classes != class_hat).int().sum(axis=-1).nonzero().shape[0]
        acc = (x.shape[0] - errors)/x.shape[0]*100


        train_loss.append(loss_out.item())
        train_acc.append(acc)
    
    model.eval()
    for x, y in zip(H_test, labels_test):
        final_len = (x == 2).nonzero().squeeze().item()+1

        x = x[:final_len].float().unsqueeze(-1)
        y_ = y[:final_len].long()-1

        logits = model(x)

        loss_out = loss(logits, y_)

        class_hat = torch.round(nn.functional.softmax(logits, dim=-1))
        classes = nn.functional.one_hot(y_)

        errors = (classes != class_hat).int().sum(axis=-1).nonzero().shape[0]
        acc = (x.shape[0] - errors)/x.shape[0]*100


        test_loss.append(loss_out.item())
        test_acc.append(acc)

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}/{epochs}. Training Loss: {round(sum(train_loss)/len(train_loss), 3)}. Training Accuracy: {round(sum(train_acc)/len(train_acc), 3)}. Test Loss: {round(sum(test_loss)/len(test_loss), 3)}. Test Accuracy: {round(sum(test_acc)/len(test_acc), 3)}.")

Epoch: 0/4000. Training Loss: 2.537. Training Accuracy: 0.0. Test Loss: 2.493. Test Accuracy: 0.0.
Epoch: 100/4000. Training Loss: 1.698. Training Accuracy: 8.919. Test Loss: 1.608. Test Accuracy: 11.806.
Epoch: 200/4000. Training Loss: 1.508. Training Accuracy: 18.915. Test Loss: 1.385. Test Accuracy: 23.611.
Epoch: 300/4000. Training Loss: 1.338. Training Accuracy: 21.785. Test Loss: 1.201. Test Accuracy: 29.514.
Epoch: 400/4000. Training Loss: 1.193. Training Accuracy: 27.797. Test Loss: 1.071. Test Accuracy: 35.764.
Epoch: 500/4000. Training Loss: 1.086. Training Accuracy: 42.079. Test Loss: 0.959. Test Accuracy: 50.347.
Epoch: 600/4000. Training Loss: 1.007. Training Accuracy: 46.693. Test Loss: 0.863. Test Accuracy: 61.806.
Epoch: 700/4000. Training Loss: 0.94. Training Accuracy: 49.497. Test Loss: 0.8. Test Accuracy: 64.583.
Epoch: 800/4000. Training Loss: 0.885. Training Accuracy: 52.006. Test Loss: 0.775. Test Accuracy: 70.833.
Epoch: 900/4000. Training Loss: 0.833. Training A

In [158]:
sentence = data["sentences"][0][0].item().split(" ")
sentence_idx = H_train[0][:8]
labels = labels_train[0][:8]
labels_hat = torch.round(nn.functional.softmax(model(sentence_idx.float().unsqueeze(-1)), dim=-1))

print("Word; Word Id; Label; Label Hat")
for word, word_id, label, label_hat in zip(sentence, sentence_idx, labels, labels_hat):
    print(word, word_id.item(), label.item(), (label_hat==1).nonzero().item()+1)

Word; Word Id; Label; Label Hat
START 3 1 1
It 48 5 5
was 77 3 3
a 4 2 2
great 36 6 6
play 58 4 4
. 1 10 10
END 2 11 11


In [199]:
data["sentiment_test"]

array([[0],
       [1],
       [0],
       [1]], dtype=uint8)

In [203]:
labels_train_sentiment = torch.tensor([sentence.item() for sentence in data["sentiment_train"]])
labels_test_sentiment = torch.tensor([sentence.item() for sentence in data["sentiment_test"]])

In [204]:
labels_test_sentiment

tensor([0, 1, 0, 1])

In [163]:
class SentimentAnalysis(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(
            input_size=1, hidden_size=features_size, num_layers=1
        )
        self.linear = nn.Linear(features_size, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x_ = self.rnn(x)
        logits = self.linear(x_[0][-1])
        y = self.sigmoid(logits)

        return y

In [215]:
model_sentiment = SentimentAnalysis()
lr = 1e-5
optimizer = torch.optim.Adam(model_sentiment.parameters(), lr=lr)
loss = nn.BCELoss()
epochs = 1000

In [216]:
for epoch in range(epochs):
    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []
    model_sentiment.train()
    for x, y in zip(H_train, labels_train_sentiment):
        final_len = (x == 2).nonzero().squeeze().item()+1

        x = x[:final_len].float().unsqueeze(-1)

        y_hat = model_sentiment(x)

        loss_out = loss(y_hat, y.unsqueeze(-1).float())
        loss_out.backward()
        optimizer.step()

        class_hat = torch.round(y_hat)

        errors = (y != class_hat).int().sum(axis=-1).nonzero().shape[0]
        acc = (x.shape[-1] - errors)/x.shape[-1]*100


        train_loss.append(loss_out.item())
        train_acc.append(acc)
    
    model_sentiment.eval()
    for x, y in zip(H_test, labels_test_sentiment):
        final_len = (x == 2).nonzero().squeeze().item()+1

        x = x[:final_len].float().unsqueeze(-1)

        y_hat = model_sentiment(x)

        loss_out = loss(y_hat, y.unsqueeze(-1).float())

        class_hat = torch.round(y_hat)

        errors = (y != class_hat).int().sum(axis=-1).nonzero().shape[0]
        acc = (x.shape[-1] - errors)/x.shape[-1]*100


        test_loss.append(loss_out.item())
        test_acc.append(acc)

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}/{epochs}. Training Loss: {round(sum(train_loss)/len(train_loss), 3)}. Training Accuracy: {round(sum(train_acc)/len(train_acc), 3)}. Test Loss: {round(sum(test_loss)/len(test_loss), 3)}. Test Accuracy: {round(sum(test_acc)/len(test_acc), 3)}.")

Epoch: 0/1000. Training Loss: 0.702. Training Accuracy: 46.667. Test Loss: 0.697. Test Accuracy: 50.0.
Epoch: 100/1000. Training Loss: 0.593. Training Accuracy: 66.667. Test Loss: 0.561. Test Accuracy: 75.0.
Epoch: 200/1000. Training Loss: 0.343. Training Accuracy: 93.333. Test Loss: 0.365. Test Accuracy: 75.0.
Epoch: 300/1000. Training Loss: 0.053. Training Accuracy: 100.0. Test Loss: 0.239. Test Accuracy: 75.0.
Epoch: 400/1000. Training Loss: 0.004. Training Accuracy: 100.0. Test Loss: 0.598. Test Accuracy: 75.0.
Epoch: 500/1000. Training Loss: 0.001. Training Accuracy: 100.0. Test Loss: 1.002. Test Accuracy: 75.0.
Epoch: 600/1000. Training Loss: 0.0. Training Accuracy: 100.0. Test Loss: 1.04. Test Accuracy: 75.0.
Epoch: 700/1000. Training Loss: 0.0. Training Accuracy: 100.0. Test Loss: 1.091. Test Accuracy: 75.0.
Epoch: 800/1000. Training Loss: 0.0. Training Accuracy: 100.0. Test Loss: 1.247. Test Accuracy: 75.0.
Epoch: 900/1000. Training Loss: 0.0. Training Accuracy: 100.0. Test Lo

In [186]:
sentence = data["sentences"][0][0].item()
sentence_idx = H_train[0][:8]
label = labels_train_sentiment[0]
labels_hat = torch.round(model_sentiment(sentence_idx.float().unsqueeze(-1)))

In [217]:
for i in range(4):
    sentence = data["sentences"][0][i].item()
    final_len = (H_train[i] == 2).nonzero().squeeze().item()+1
    sentence_idx = H_train[i][:final_len]
    label = labels_train_sentiment[i]
    label_hat = torch.round(model_sentiment(sentence_idx.float().unsqueeze(-1)))
    print(sentence, label.item(), label_hat.item())

START It was a great play . END 1 1.0
START I had a bad experience with the rental car . END 0 0.0
START Her flight was delayed for over five hours . END 0 0.0
START She passed her exam easily . END 1 1.0


In [231]:
class SentenceAnalysisLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.LSTM(
            input_size=1, hidden_size=features_size, num_layers=1
        )
        self.linear = nn.Linear(features_size, word_types)
    
    def forward(self, x):
        x_ = self.rnn(x)
        logits = self.linear(x_[0])

        return logits

In [232]:
lstm_model = SentenceAnalysisLSTM()
lr = 1e-5
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()
epochs = 4000

In [229]:
for epoch in range(epochs):
    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []
    model.train()
    for x, y in zip(H_train, labels_train):
        final_len = (x == 2).nonzero().squeeze().item()+1

        x = x[:final_len].float().unsqueeze(-1)
        y_ = y[:final_len].long()-1

        logits = lstm_model(x)

        loss_out = loss(logits, y_)
        loss_out.backward()
        optimizer.step()

        class_hat = torch.round(nn.functional.softmax(logits, dim=-1))
        classes = nn.functional.one_hot(y_)

        errors = (classes != class_hat).int().sum(axis=-1).nonzero().shape[0]
        acc = (x.shape[0] - errors)/x.shape[0]*100


        train_loss.append(loss_out.item())
        train_acc.append(acc)
    
    model.eval()
    for x, y in zip(H_test, labels_test):
        final_len = (x == 2).nonzero().squeeze().item()+1

        x = x[:final_len].float().unsqueeze(-1)
        y_ = y[:final_len].long()-1

        logits = lstm_model(x)

        loss_out = loss(logits, y_)

        class_hat = torch.round(nn.functional.softmax(logits, dim=-1))
        classes = nn.functional.one_hot(y_)

        errors = (classes != class_hat).int().sum(axis=-1).nonzero().shape[0]
        acc = (x.shape[0] - errors)/x.shape[0]*100


        test_loss.append(loss_out.item())
        test_acc.append(acc)

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}/{epochs}. Training Loss: {round(sum(train_loss)/len(train_loss), 3)}. Training Accuracy: {round(sum(train_acc)/len(train_acc), 3)}. Test Loss: {round(sum(test_loss)/len(test_loss), 3)}. Test Accuracy: {round(sum(test_acc)/len(test_acc), 3)}.")

Epoch: 0/4000. Training Loss: 2.419. Training Accuracy: 0.0. Test Loss: 2.399. Test Accuracy: 0.0.
Epoch: 100/4000. Training Loss: 2.01. Training Accuracy: 0.0. Test Loss: 1.973. Test Accuracy: 0.0.
Epoch: 200/4000. Training Loss: 1.734. Training Accuracy: 10.131. Test Loss: 1.642. Test Accuracy: 11.806.
Epoch: 300/4000. Training Loss: 1.57. Training Accuracy: 19.656. Test Loss: 1.471. Test Accuracy: 23.611.
Epoch: 400/4000. Training Loss: 1.447. Training Accuracy: 19.656. Test Loss: 1.35. Test Accuracy: 23.611.
Epoch: 500/4000. Training Loss: 1.338. Training Accuracy: 23.09. Test Loss: 1.228. Test Accuracy: 26.389.
Epoch: 600/4000. Training Loss: 1.245. Training Accuracy: 25.826. Test Loss: 1.114. Test Accuracy: 29.167.
Epoch: 700/4000. Training Loss: 1.161. Training Accuracy: 31.281. Test Loss: 1.027. Test Accuracy: 40.972.
Epoch: 800/4000. Training Loss: 1.081. Training Accuracy: 32.443. Test Loss: 0.955. Test Accuracy: 44.097.
Epoch: 900/4000. Training Loss: 1.01. Training Accuracy

In [230]:
sentence = data["sentences"][0][0].item().split(" ")
sentence_idx = H_train[0][:8]
labels = labels_train[0][:8]
labels_hat = torch.round(nn.functional.softmax(lstm_model(sentence_idx.float().unsqueeze(-1)), dim=-1))

print("Word; Word Id; Label; Label Hat")
for word, word_id, label, label_hat in zip(sentence, sentence_idx, labels, labels_hat):
    print(word, word_id.item(), label.item(), (label_hat==1).nonzero().item()+1)

Word; Word Id; Label; Label Hat
START 3 1 1
It 48 5 5
was 77 3 3
a 4 2 2
great 36 6 6
play 58 4 4
. 1 10 10
END 2 11 11


In [236]:
class SentenceAnalysisBiderectionalLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.LSTM(
            input_size=1, hidden_size=features_size, num_layers=1, bidirectional=True
        )
        self.linear = nn.Linear(features_size*2, word_types)
    
    def forward(self, x):
        x_ = self.rnn(x)
        logits = self.linear(x_[0])

        return logits

In [246]:
bidirectional_lstm_model = SentenceAnalysisBiderectionalLSTM()
lr = 1e-5
bidirectional_lstm_optimizer = torch.optim.Adam(bidirectional_lstm_model.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()
epochs = 4000

In [247]:
for epoch in range(epochs):
    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []
    model.train()
    for x, y in zip(H_train, labels_train):
        final_len = (x == 2).nonzero().squeeze().item()+1

        x = x[:final_len].float().unsqueeze(-1)
        y_ = y[:final_len].long()-1

        logits = bidirectional_lstm_model(x)

        loss_out = loss(logits, y_)
        loss_out.backward()
        bidirectional_lstm_optimizer.step()

        class_hat = torch.round(nn.functional.softmax(logits, dim=-1))
        classes = nn.functional.one_hot(y_)

        errors = (classes != class_hat).int().sum(axis=-1).nonzero().shape[0]
        acc = (x.shape[0] - errors)/x.shape[0]*100


        train_loss.append(loss_out.item())
        train_acc.append(acc)
    
    model.eval()
    for x, y in zip(H_test, labels_test):
        final_len = (x == 2).nonzero().squeeze().item()+1

        x = x[:final_len].float().unsqueeze(-1)
        y_ = y[:final_len].long()-1

        logits = bidirectional_lstm_model(x)

        loss_out = loss(logits, y_)

        class_hat = torch.round(nn.functional.softmax(logits, dim=-1))
        classes = nn.functional.one_hot(y_)

        errors = (classes != class_hat).int().sum(axis=-1).nonzero().shape[0]
        acc = (x.shape[0] - errors)/x.shape[0]*100


        test_loss.append(loss_out.item())
        test_acc.append(acc)

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}/{epochs}. Training Loss: {round(sum(train_loss)/len(train_loss), 3)}. Training Accuracy: {round(sum(train_acc)/len(train_acc), 3)}. Test Loss: {round(sum(test_loss)/len(test_loss), 3)}. Test Accuracy: {round(sum(test_acc)/len(test_acc), 3)}.")

Epoch: 0/4000. Training Loss: 2.37. Training Accuracy: 0.0. Test Loss: 2.385. Test Accuracy: 0.0.
Epoch: 100/4000. Training Loss: 1.763. Training Accuracy: 0.0. Test Loss: 1.704. Test Accuracy: 0.0.
Epoch: 200/4000. Training Loss: 1.386. Training Accuracy: 29.18. Test Loss: 1.271. Test Accuracy: 35.417.
Epoch: 300/4000. Training Loss: 1.191. Training Accuracy: 30.898. Test Loss: 1.072. Test Accuracy: 38.194.
Epoch: 400/4000. Training Loss: 1.057. Training Accuracy: 41.152. Test Loss: 0.956. Test Accuracy: 55.903.
Epoch: 500/4000. Training Loss: 0.955. Training Accuracy: 45.817. Test Loss: 0.864. Test Accuracy: 55.903.
Epoch: 600/4000. Training Loss: 0.876. Training Accuracy: 49.363. Test Loss: 0.81. Test Accuracy: 62.153.
Epoch: 700/4000. Training Loss: 0.81. Training Accuracy: 52.242. Test Loss: 0.764. Test Accuracy: 62.153.
Epoch: 800/4000. Training Loss: 0.751. Training Accuracy: 55.028. Test Loss: 0.74. Test Accuracy: 62.153.
Epoch: 900/4000. Training Loss: 0.697. Training Accuracy

In [248]:
sentence = data["sentences"][0][0].item().split(" ")
sentence_idx = H_train[0][:8]
labels = labels_train[0][:8]
labels_hat = torch.round(nn.functional.softmax(bidirectional_lstm_model(sentence_idx.float().unsqueeze(-1)), dim=-1))

print("Word; Word Id; Label; Label Hat")
for word, word_id, label, label_hat in zip(sentence, sentence_idx, labels, labels_hat):
    print(word, word_id.item(), label.item(), (label_hat==1).nonzero().item()+1)

Word; Word Id; Label; Label Hat
START 3 1 1
It 48 5 5
was 77 3 3
a 4 2 2
great 36 6 6
play 58 4 4
. 1 10 10
END 2 11 11
